# Indicators i6 & i7 & i8 of IRD Tuna Atlas

This notebook is available on https://github.com/juldebar/IRDTunaAtlas/blob/master/jupyter_notebook/sardara_notebook_i6i7i8.Rmd

This notebook has been written and executed on https://blue-cloud-jupyterhub.d4science.org/ 

It uses data provided by Fishereries Atlas VRE

It can be used as a documentation of underlying processes used by the related shiny app  https://github.com/juldebar/IRDTunaAtlas/blob/master/shiny/sardara_shiny_i6i7i8.Rmd

## Load R packages

In [ ]:
library(DBI)
library(sp)
library(sf)
library(plyr)
library(dplyr)
library(ggplot2)
library(plotly)
install.packages("maps")
library(maps)

## 1. Connect the database

In [5]:
#######################
##Database Identifiers: tunaatlas / db-tunaatlas
source(file = "credentials.R")
con <- dbConnect(drv=DRV,  dbname = Dbname, host = Host , user=User, password=Password,port="5432")
source("https://raw.githubusercontent.com/juldebar/IRDTunaAtlas/master/R/TunaAtlas_i6_SpeciesMap.R")
source("https://raw.githubusercontent.com/juldebar/IRDTunaAtlas/master/R/TunaAtlas_i7_SpeciesMapRelativeCatches.R")
source("https://raw.githubusercontent.com/juldebar/IRDTunaAtlas/master/R/TunaAtlas_i8_SpeciesMapRelativeCatchesOtherSpecies.R")

## 2. Execute SQL query to get data

In [ ]:
new_wkt <- 'POLYGON((-180 -90, 180 -90, 180 90, -180 90, -180 -90))'
wkt <- new_wkt

target_species <- c("YFT","BET","SKJ")
target_year <- c(seq(1:10)+1994)
#query_i6i7i8 <- paste0("SELECT ogc_fid, geom_id, geom, year, species, country, value, count FROM i6i7i8 WHERE species IN ('",paste0(target_species,collapse="','"),"') AND year IN ('",paste0(target_year,collapse="','"),"');")
query_i6i7i8 <- paste0("SELECT ogc_fid, geom_id, geom, year, species, country, value, count FROM i6i7i8;")


df_i6i7i8 = st_read(con, query = query_i6i7i8)
all_years = unique(df_i6i7i8$year)
all_species = unique(df_i6i7i8$species)
all_years
all_species

## 3. Plot data with indicator 8

In [ ]:
df_i6i7i8_filtered <- df_i6i7i8 %>% filter(year %in% target_year, species %in% target_species)
df_i8_filtered <- as(df_i6i7i8_filtered, "Spatial")

i8 <- Atlas_i8_SpeciesMapRelativeCatchesOtherSpecies(df=df_i8_filtered,
                                             targetedSpecies=target_species[1],
                                             geomIdAttributeName="geom_id",
                                             yearAttributeName="year",
                                             speciesAttributeName="species",
                                             valueAttributeName="value",
                          withSparql=FALSE
                          )

i8 

## 4. Plot data with indicator 7

In [ ]:
df_i6i7i8_filtered <- df_i6i7i8 %>% filter(year == 2000, species == 'YFT')
df_i7_filtered <- as(df_i6i7i8_filtered, "Spatial")

i7 <-  Atlas_i7_SpeciesMapRelativeCatches(df=df_i7_filtered, 
                              geomIdAttributeName="geom_id",
                              yearAttributeName="year", 
                              speciesAttributeName="species",                                         
                              valueAttributeName="value",
                          withSparql=FALSE
                          )
    
i7

## 5. Plot data with indicator 6

In [ ]:
df_i6i7i8_filtered <- df_i6i7i8 %>% filter(year == 2002, species == 'YFT')
df_i6_filtered <- as(df_i6i7i8_filtered, "Spatial")

i6 <- Atlas_i6_SpeciesMap(df=df_i6_filtered,
                          geomIdAttributeName="geom_id",
                          yearAttributeName="year",
                          speciesAttributeName="species",
                          valueAttributeName="value",
                          withSparql=FALSE
                          )
i6